In [ ]:
-- Set query tag for tracking
ALTER SESSION SET query_tag = '{"origin":"sf_sit-is","name":"quantitative_research_with_ai_functions_and_cortex_code","version":{"major":1,"minor":0},"attributes":{"is_quickstart":1,"source":"notebook"}}';

# Cortex for Financial Services

Snowflake has been the trusted data platform for our financial services customers, and we are excited to showcase all our latest AI functionality to further supercharge your workflow. In this notebook, we will:

1. **Extract Analyst Sentiment** - Use Cortex AI SQL (`AI_COMPLETE`) to process earnings call transcripts
2. **Create Cortex Search** - Enable semantic search over sentiment insights
3. **Create Semantic View** - Enable natural language queries with Cortex Analyst

## 📝 Cortex AISQL: Turn your Unstructured data into Structured


**AI-based sentiment dataset** for the **Dow Jones 30 earnings calls** (2024+).
- **Source**: annotated Q&A transcripts from `COMPANY_EVENT_TRANSCRIPT_ATTRIBUTES_V2`.  
- **AI Scoring**: Claude (`claude-4-sonnet`) analyzed only **analyst questions/tone**, ignoring management’s scripted remarks.  
- **Output**: JSON with  
  - `sentiment_score` (1–10 scale),  
  - `reason` (brief justification),  
  - `analyst_count` (unique analysts).  

In [ ]:
-- create analyst sentiment for DOW JONES 30 company 
-- Note timestamp is in UTC

create or replace table ai_transcripts_analysts_sentiments AS (
WITH 
ai_analysis AS (
    SELECT
        primary_ticker,
        event_timestamp,
        event_type,
        created_at,
        ai_complete(
            'claude-4-sonnet', -- use claude for the moment 
            CONCAT_WS('\n',
                'You are analyzing sell side and buy side analysts sentiment in a public company''s earnings call transcript.',
                'Focus ONLY on analyst questions, tone, and reactions to management responses.- Ignore management’s prepared remarks unless directly referenced by analysts. - Pay attention to how analysts compare results to prior earnings calls and to market expectations.',
                'Use a CONSISTENT STANDARD across all earnings calls, regardless of sector, size, or company specifics.',
                'Evaluate (internally, equal weight) the following aspects:',
                'a) Guidance/Financial Outlook (revenue growth, earnings, margins, bookings, cash flow)',
                'b) Strategy/Product/Innovation (new initiatives, products, technologies, or business models)',
                'c) Competitive/Market Positioning (market share, competition, regulation, macro/sector trends)',
                'd) Management Execution & Credibility (track record, transparency, quality of responses, consistency with prior guidance)',
                'Compute ONE overall analyst sentiment score on a 1-10 scale, where:',
                '1 = Extremely pessimistic/concerned, 5 = Neutral/mixed, 10 = Extremely optimistic/bullish.',
                'Return your response as valid JSON with this exact format:',
                '{"score": <integer 1-10>, "reason": <brief explanation covering the four aspects>, "analyst_count": <integer number of unique analysts>}',
                'Transcript:',
                transcript
            )
        ) AS ai_response
    FROM unique_transcripts
)

SELECT
    primary_ticker,
    event_timestamp,
    event_type,
    created_at,
    /* Safely parse JSON. If parsing fails, these will be NULL. */
    (TRY_PARSE_JSON(ai_response):score)::INT      AS sentiment_score,
    (TRY_PARSE_JSON(ai_response):analyst_count)::INT AS unique_analyst_count,
    (TRY_PARSE_JSON(ai_response):reason)::STRING  AS sentiment_reason
FROM ai_analysis
-- where unique_analyst_count > 1 -- remove data that only contains speakers from the public company
order by primary_ticker, event_timestamp
);

select * from ai_transcripts_analysts_sentiments;

## ✅ Notebook Complete!

You have successfully **extracted analyst sentiment** from earnings call transcripts using Cortex AI Functions.

### Next Steps
1. Open the **TRAIN_ML_MODELS** notebook to train and register ML prediction models
2. Run **scripts/create_cortex_components.sql** to create Cortex Search, Semantic View, and Agent
3. Test the agent in **Snowflake Intelligence**